# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Métodos de Regresión</center>
### <center> Práctica Regresión Logística </center>

#### Links interesantes
 * pandas.pydata.org
 * scikit-yb.org
 * matplotlib.org
 * seaborn.pydata.org
 * scikit-yb.org


Importamos librerías generales

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Importamos librerías de modelos

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import cross_val_score,cross_validate

Importamos libreía de métricas

In [3]:
#Metricas para evaluar modelos
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

from yellowbrick.classifier import DiscriminationThreshold
from yellowbrick.classifier import ROCAUC

### Ejercicio 1: Clasificación Binaria

Cargamos el dataset sobre el cual vamos a trabajar

In [5]:
directorio="../datasets"
archivo="ds_titanic.csv"
ds_train=pd.read_csv(directorio+"/"+archivo)
ds_trabajo=ds_train.copy()

FileNotFoundError: ignored

## Análisis exploratorio

Listamos la cantidad de filas y columnas que tiene el dataset

In [ ]:
ds_trabajo.shape

Listamos los nombres de las columnas del dataset

In [ ]:
ds_trabajo.columns.tolist()

<b>Descripción de los campos</b><br>
PassengerId: Identificador del pasajero dentro de la tabla<br>
Survived: Indica si el pasajero sobrevivió<br>
Pclass: clase de pasajero primera(1), segunda (2) o tercera (3) <br>
Name: Nombre del pasajero <br>
Sex: Sexo del pasajero<br>
Age: Edad<br>
SibSp: Cantidad de hermanas/os o esposas/os a bordo<br>
Parch: Cantidad de padres o hijos a bordo<br>
Ticket: Número de ticket<br>
Fate: Tarifa <br>
Cabin: Cabina en la cual se encuentra alojado el pasajero<br>
Embarked: Puerta de embarque<br>

Mostramos algunas filas del dataframe

In [ ]:
ds_trabajo.head()


### <font color='red'>To do (tarea) </font>
##### <font color='red'> Continuar con el análisis exploratorio  </font>    

## Preprocesamiento

<font color='blue'> A&uacute;n no explicamos todos los conceptos necesarios para realizar un preprocesamiento de datos exhaustivo.  <br>
    (paciencia, esto lo vas a ir adquiriendo a lo largo del curso) </font>  <br>
<font color='blue'> C&oacute;mo comentamos en clase, vamos a realizar muchas simplificaciones </font>

Eliminamos algunas columnas que decidimos no utilizar

In [ ]:
columnas_eliminar=['PassengerId','Name',"Ticket"]
ds_trabajo.drop(columnas_eliminar, axis='columns', inplace=True)  #Elimina columna sobre el mismo dataframe
ds_trabajo.columns.tolist()

In [ ]:
#Grafico un pairplot
sns.pairplot(ds_trabajo)
plt.show()

#### Datos nulos

In [ ]:
filas_totales=ds_trabajo.shape[0]
print(ds_trabajo.isna().sum()/filas_totales*100)

<font color='blue'>Cómo aún no aprendimos como tratar datos nulos, vamos eliminar los valores faltantes. </font> <br> <br>
 Debido a la gran cantidad de datos nulos en la variable Cabin se decide eliminarla. <br>
Para el resto de las variables con datos faltantes se eliminaran las filas que tengan valores nulos en alguna de ellas.

In [ ]:
columnas_eliminar_NANs=['Cabin']
ds_trabajo.drop(columnas_eliminar_NANs, axis='columns', inplace=True)
ds_trabajo.columns.tolist()

In [ ]:
ds_trabajo=ds_trabajo.dropna()

Volvemos a listar la cantidad de Nans

In [ ]:
filas_totales=ds_trabajo.shape[0]
print(ds_trabajo.isna().sum()/filas_totales*100)

In [ ]:
ds_trabajo.shape

Continuamos trabajando con un dataset de 712 filas y 8 variables

<font color='blue'> Muchas veces para trabajar con determinados m&eacute;todos es necesario realizar transformaciones de variables.</font> <br>
<font color='blue'> En este caso al utilizar Regresión Logística, y querer utilizar variables categ&oacute;ricas como variables predictoras, necesitamos realizar transformaciones sobre las mismas </font> <br>
<font color='blue'> Este tema lo vamor a retomar en la clase de Ingenier&iacute;a de Caracter&iacute;sticas. <br>
    Por ahora te contamos que cada variable categ&oacute;rica la transformamos en tantas variables dummies como valores distintos pueda tomar. </font>

In [ ]:
ds_trabajo = pd.get_dummies(ds_trabajo, columns=["Pclass","Sex","Embarked"])
ds_trabajo.head()

Buscamos filas repetidas

In [ ]:
ds_trabajo_dup = ds_trabajo[ds_trabajo.duplicated(keep=False)]
ds_trabajo_dup.head()

In [ ]:
ds_trabajo.drop_duplicates(inplace=True)

In [ ]:
ds_trabajo.shape

#### Divisi&oacute;n Train - Test

Dividimos en conjunto de datos inicial, en dos subconjuntos. <br>
Uno destinado a entrenar el modelo y otro para probarlo sobre datos que no vio <br>
En este caso vamos a entrenar utilizando el 70% de los datos y validando los modelos sobre el 20% de datos restantes.

In [ ]:
columnas_modelo = ds_trabajo.columns.tolist()
columnas_modelo.remove('Survived')

ds_trabajo_x = ds_trabajo[columnas_modelo].copy()
ds_trabajo_y = ds_trabajo['Survived'].copy()

x_train, x_test, y_train, y_test = train_test_split(ds_trabajo_x, ds_trabajo_y, test_size=0.2, random_state=2)

#### Comparamos la distribución de la variable Survived

In [ ]:
ds_agrupado_survived=ds_agrupado_Pclass=ds_trabajo.groupby(['Survived']).count()  #Calculo la frecuencia de cada categoría
cant_total=ds_trabajo.shape[0]

## Declaramos valores para los ejes
eje_x = ds_agrupado_survived.index.tolist()
eje_y = [i/cant_total for i in ds_agrupado_survived['Age'].tolist()]

plt.figure(figsize=(5, 4))
sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_survived ,alpha=0.5).set(title="Frecuencia relativa en variable objetivo - Dataset completo",ylabel='Frecuencia relativa',xlabel='Survived')
plt.show()

In [ ]:
ds_aux=pd.DataFrame(list(zip(y_train)),columns=["Survived"]).reset_index()
ds_agrupado_survived=ds_aux.groupby(['Survived']).count()  #Calculo la frecuencia de cada categoría
cant_total=ds_aux.shape[0]

## Declaramos valores para los ejes
eje_x = ds_agrupado_survived.index.tolist()  
eje_y = [ i/cant_total for i in ds_agrupado_survived['index'].tolist()]

plt.figure(figsize=(5, 4))
sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_survived ,alpha=0.5).set(title="Frecuencia relativa en variable objetivo - Dataset train",ylabel='Frecuencia relativa',xlabel='Survived')
plt.show()

In [ ]:
ds_aux=pd.DataFrame(list(zip(y_test)),columns=["Survived"]).reset_index()

ds_agrupado_survived=ds_aux.groupby(['Survived']).count()  #Calculo la frecuencia de cada categoría
ds_agrupado_survived.head()

cant_total=y_test.shape[0]
eje_x = ds_agrupado_survived.index.tolist()   ## Declaramos valores para el eje x
eje_y = [ i/cant_total for i in ds_agrupado_survived['index'].tolist()]  ## Declaramos valores para el eje y

plt.figure(figsize=(5, 4))
sns_bar_plot=sns.barplot(x=eje_x, y=eje_y, data=ds_agrupado_survived ,alpha=0.5).set(title="Frecuencia relativa en variable objetivo - Dataset test",ylabel='Frecuencia relativa',xlabel='Survived')
plt.show()

### <font color='red'>To do (tarea)</font>
##### <font color='red'> Evaluar c&oacute;mo se comportan los modelos propuestos a continuaci&oacute;n con distintas segmentaciones del dataset original en train y test  </font>    
##### <font color='red'> Probar por ejemplo: </font>   
* ##### <font color='red'> Desbalance de clases en entrenamiento (por ej 90% de las observaciones correspondiente a la clase mayoritaria) </font>   
* ##### <font color='red'> Igual cantidad de muestras en ambas clases </font>   
* ##### <font color='red'> Mayor cantidad de observaciones de la clase minoritaria </font>   

## Clasificación

##### Vamos a proponer 2 modelos

#### Modelo 1
Realizamos un modelo de regresión logística para predecir la supervivencia en función de Pclass, Sex y Age.

In [ ]:
columnas_modelo1 = ['Pclass_1','Pclass_2','Sex_female','Age']
x_train_modelo1 = x_train[columnas_modelo1].copy()
y_train_modelo1 = y_train.copy()

x_test_modelo1 = x_test[columnas_modelo1].copy()
y_test_modelo1 = y_test.copy()

 Entrenamos el modelo

In [ ]:
modelo1 = LogisticRegression()
modelo1.fit(x_train_modelo1, y_train_modelo1)

Evaluamos al modelo sobre datos de test

In [ ]:
y_pred_modelo1 = modelo1.predict(x_test_modelo1)
y_pred_proba_modelo1 = modelo1.predict_proba(x_test_modelo1)[:, 1]

Existen varias m&eacute;tricas para evaluar modelos <br>
Primero miremos una tabla de contingencia

In [ ]:
ds_cm=pd.DataFrame(list(zip(y_test_modelo1, y_pred_proba_modelo1)),columns=["Survived","Prob"])
ds_cm["Survived_predict"]=0
ds_cm.loc[ds_cm.loc[:,"Prob"]>=0.50,"Survived_predict"]=1

tabla=pd.crosstab(ds_cm['Survived'], ds_cm['Survived_predict'])
plt.figure(figsize=(5, 4))
grf=sns.heatmap(tabla,annot=True, cmap = 'Blues')
plt.show()

Utilicemos AUC

In [ ]:
visualizer = ROCAUC(modelo1,binary=True)

plt.figure(figsize=(5, 4))
visualizer.fit(x_train_modelo1, y_train_modelo1)
visualizer.score(x_test_modelo1, y_test_modelo1)

grf=visualizer.show()

### Sumemos validación cruzada
Estamos entrenando con un solo conjunto de datos, probemos usando validación cruzada

In [ ]:
modelo_cv = LogisticRegressionCV(cv=5,scoring= 'roc_auc')
modelo_cv.fit(x_train_modelo1, y_train_modelo1)

In [ ]:
y_pred_modelo_cv = modelo_cv.predict(x_test_modelo1)
y_pred_proba_modelo_cv = modelo_cv.predict_proba(x_test_modelo1)[:, 1]

In [ ]:
ds_vc=pd.DataFrame(list(zip(y_test_modelo1, y_pred_proba_modelo_cv)),columns=["Survived","Prob"])
ds_vc["Survived_predict"]=0
ds_vc.loc[ds_cm.loc[:,"Prob"]>=0.50,"Survived_predict"]=1

tabla=pd.crosstab(ds_vc['Survived'], ds_vc['Survived_predict'])
plt.figure(figsize=(5, 4))
grf=sns.heatmap(tabla,annot=True, cmap = 'Blues')
plt.show()

In [ ]:
visualizer = ROCAUC(modelo_cv,binary=True)

plt.figure(figsize=(5, 4))
visualizer.fit(x_train_modelo1, y_train_modelo1)
visualizer.score(x_test_modelo1, y_test_modelo1)

grf=visualizer.show()

#### Modelo 2
Realizamos un modelo de regresión logística para predecir la supervivencia en función de Sex y la puerta de embarque.

In [ ]:
columnas_modelo2 = ['Sex_female','Embarked_C','Embarked_Q','Embarked_S']
x_train_modelo2 = x_train[columnas_modelo2].copy()
y_train_modelo2 = y_train.copy()

x_test_modelo2 = x_test[columnas_modelo2].copy()
y_test_modelo2 = y_test.copy()

In [ ]:
modelo2 = LogisticRegression()
modelo2.fit(x_train_modelo2, y_train_modelo2)

y_pred_modelo2 = modelo2.predict(x_test_modelo2)
y_pred_proba_modelo2 = modelo2.predict_proba(x_test_modelo2)[:, 1]

In [ ]:
visualizer = ROCAUC(modelo2,binary=True)

plt.figure(figsize=(5, 4))
visualizer.fit(x_train_modelo2, y_train_modelo2)
visualizer.score(x_test_modelo2, y_test_modelo2)
grf=visualizer.show()

Sumamos validación cruzada

In [ ]:
modelo_cv2 = LogisticRegressionCV(cv=5,scoring= 'roc_auc')
modelo_cv2.fit(x_train_modelo2, y_train_modelo2)

In [ ]:
modelo_cv2.scores_[1].mean()

In [ ]:
y_pred_modelo_cv2 = modelo_cv2.predict(x_test_modelo2)
y_pred_proba_modelo_cv2 = modelo_cv2.predict_proba(x_test_modelo2)[:, 1]

In [ ]:
ds_cv2=pd.DataFrame(list(zip(y_test_modelo2, y_pred_proba_modelo_cv2)),columns=["Survived","Prob"])
ds_cv2["Survived_predict"]=0
ds_cv2.loc[ds_cm.loc[:,"Prob"]>=0.50,"Survived_predict"]=1

tabla=pd.crosstab(ds_cv2['Survived'], ds_cv2['Survived_predict'])
plt.figure(figsize=(5, 4))
grf2=sns.heatmap(tabla,annot=True, cmap = 'Blues')
plt.show()

In [ ]:
metrics.accuracy_score(ds_cv2.Survived, ds_cv2.Survived_predict)

In [ ]:
metrics.f1_score(ds_cv2.Survived, ds_cv2.Survived_predict)

In [ ]:
metrics.recall_score(ds_cv2.Survived, ds_cv2.Survived_predict)

In [ ]:
metrics.precision_score(ds_cv2.Survived, ds_cv2.Survived_predict)

### Ejercicio 1: Clasificación multiclase

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

In [ ]:
print(iris['feature_names'])

In [ ]:
nombres_columnas=[ x.replace('(cm)','').rstrip().replace(" ","_") for x in iris['feature_names']]

df_iris=pd.DataFrame(iris['data'],columns=nombres_columnas)
y_iris=iris['target']
df_iris.head()

<img src="https://www.pngkey.com/png/detail/82-826789_iris-iris-sepal-and-petal.png" alt="Iris - Iris Sepal And Petal@pngkey.com">

## Análisis exploratorio

In [ ]:
plt.figure(figsize=(20, 6))

for f, f_name in enumerate(iris['feature_names']):
    plt.subplot(1,4,f+1)
    data = np.zeros((50,3))
    for k in range(3): #
        data[:,k] = iris.data[iris.target==k,f]      
    plt.violinplot(data)
    plt.ylabel(str(f)+' '+ f_name)
    plt.xticks([1,2,3], labels=iris['target_names'], rotation = 70)

plt.show() 

Se puede distinguir que la categoría Setosa se puede distinguir del resto utilizando dos atributos. <br>
Sin embargo Versicolor y Virnica no son tan fácilmente separables

In [ ]:
color_map = {-1: (1, 1, 1), 0: (0, 0, .9), 1: (1, 0, 0), 2: (.5, .5, 0)}
colors = [color_map[y] for y in iris.target]
plt.figure(figsize=(20, 8))
plt.scatter(df_iris.petal_length,df_iris.petal_width,c=colors,marker='.')
plt.xlabel('petal_length')
plt.ylabel('petal_width')
plt.show()

Pensemos un modelo que sólo incluya estas 2 variables

In [ ]:
df_iris_2_vars=df_iris[['petal_length','petal_width']]
df_iris_2_vars.head()

Aclaración importante: <br>
Esta vez no separamos en train-test porque estamos haciendo una prueba para entender cómo funciona el modelo

In [ ]:
clf_2_vars = LogisticRegression(multi_class='multinomial')
clf_2_vars.fit(df_iris_2_vars, y_iris)

In [ ]:
x1_min, x1_max = np.min(df_iris.petal_length), np.max(df_iris.petal_length)
x2_min, x2_max = np.min(df_iris.petal_width), np.max(df_iris.petal_width)
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.1),
                     np.arange(x2_min, x2_max, 0.1)) 

XX = np.c_[xx1.ravel(), xx2.ravel()]
pred = clf_2_vars.predict(XX)

In [ ]:
plt.figure(3)
pred = pred.reshape(xx1.shape)
plt.contourf(xx1, xx2, pred, cmap=plt.cm.Pastel2)

plt.scatter(df_iris_2_vars.petal_length,df_iris_2_vars.petal_width,c = colors,marker='.')
plt.title(u'División del espacio de caracterísicas por clase')
plt.xlabel(iris['feature_names'][1])
plt.ylabel(iris['feature_names'][2])
plt.show()

Creemos un modelo utilizando todas las variables

#### Ahora si separemos en train y test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( df_iris, y_iris, test_size=0.2, random_state=1)

In [ ]:
clf_iris_rl=LogisticRegressionCV(cv=5,multi_class='multinomial')

clf_iris_rl.fit(X_train, y_train)

In [ ]:
def plot_confusion_matrix(clf, X_test, y_test, y_pred, labels):
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    fig = plt.figure()
    sns.heatmap(cm, annot = True, xticklabels =labels, yticklabels = labels, cmap = 'Blues')
    plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

plt.figure(figsize=(4, 4))

y_pred = clf_iris_rl.predict(X_test)
print("Test confusion matrix:")
plot_confusion_matrix(clf_iris_rl, X_test, y_test, y_pred,labels=  iris['target_names'])

#### Vamos a ver métricas para evaluar estos casos más adelante!!